In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import json

In [2]:
data_root = Path('data')
hexses_target_path = data_root / 'hexses_target.lst'
hexses_data_path = data_root / 'hexses_data.lst'

train_data_fn   = data_root / 'transactions.parquet'
train_target_fn = data_root / 'target.parquet'

In [3]:
with open(hexses_target_path, "r") as f:
    hexses_target = [x.strip() for x in f.readlines()]

In [4]:
with open(hexses_data_path, "r") as f:
    hexses_data = [x.strip() for x in f.readlines()]

In [5]:
target = (
    pd.read_parquet(train_target_fn)
    .assign(customer_id = lambda x: x.customer_id.astype(int))
    .pipe(lambda x: pd.pivot(x.assign(v = 1.), index='customer_id', columns='h3_09', values='v'))
    .pipe(lambda x: x.reindex(sorted(x.columns), axis=1)) # Сортируем столбцы по порядку
    .sort_values(by='customer_id') # сортируем строки
    .fillna(0)
)
target.shape

(69337, 1657)

In [6]:
pd.DataFrame({"target": target.mean()}).to_json("submit/means.json")